# This notebook will be used in Coursera Capstone Project

## Importing libraries

In [281]:
import pandas as pd # for data analysis
import numpy as np # for data analysis, matrix, vector etc.
import requests # library to handle requests
from bs4 import BeautifulSoup # for webpage scrapping
import folium # map rendering library
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs

Solving environment: done

# All requested packages already installed.



In [282]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
# print("Hello Capstone Project Course!")

## Scraping Toronto ZipCode/Neighborhood Data from Wikipedia and Data Wrangling

In [283]:
# get data from Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
headers= {'User-Agent':'Mozilla/5.0'}
response = requests.get(url,headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
wikitable = soup.find('table', attrs={'class':'wikitable sortable'})
response.status_code # status_code 200 means content was downloaded correctly

200

In [284]:
wikitable

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North

In [285]:
# Identify columns and retrieve headers:
headers=wikitable.findAll('th')
for i, head in enumerate(headers): headers[i]=str(headers[i]).replace("<th>","").replace("</th>","").replace("\n","")
headers

['Postcode', 'Borough', 'Neighbourhood']

In [286]:
# Create lists for each column,
A=[]
B=[]
C=[]

for row in wikitable.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [287]:
# Clean up each list by dropping \n
A= [w.replace('\n', '') for w in A]
B= [w.replace('\n', '') for w in B]
C= [w.replace('\n', '') for w in C]

In [288]:
C

['Not assigned',
 'Not assigned',
 'Parkwoods',
 'Victoria Village',
 'Harbourfront',
 'Lawrence Heights',
 'Lawrence Manor',
 "Queen's Park",
 'Not assigned',
 'Islington Avenue',
 'Rouge',
 'Malvern',
 'Not assigned',
 'Don Mills North',
 'Woodbine Gardens',
 'Parkview Hill',
 'Ryerson',
 'Garden District',
 'Glencairn',
 'Not assigned',
 'Not assigned',
 'Cloverdale',
 'Islington',
 'Martin Grove',
 'Princess Gardens',
 'West Deane Park',
 'Highland Creek',
 'Rouge Hill',
 'Port Union',
 'Not assigned',
 'Flemingdon Park',
 'Don Mills South',
 'Woodbine Heights',
 'St. James Town',
 'Humewood-Cedarvale',
 'Not assigned',
 'Not assigned',
 'Bloordale Gardens',
 'Eringate',
 'Markland Wood',
 'Old Burnhamthorpe',
 'Guildwood',
 'Morningside',
 'West Hill',
 'Not assigned',
 'Not assigned',
 'The Beaches',
 'Berczy Park',
 'Caledonia-Fairbanks',
 'Not assigned',
 'Not assigned',
 'Not assigned',
 'Woburn',
 'Not assigned',
 'Not assigned',
 'Leaside',
 'Central Bay Street',
 'Christie'

In [289]:
# Create dataframe using each list
import pandas as pd
df=pd.DataFrame(A,columns=['Postcode'])
df['Borough']=B
df['Neighbourhood']=C

In [290]:
df.dtypes

Postcode         object
Borough          object
Neighbourhood    object
dtype: object

In [291]:
# copy Borough value to Neighborhood if NaN:
df.Neighbourhood.fillna(df.Borough, inplace=True)
df=df.drop_duplicates()

In [292]:
# Filter out rows where Borough is not assigned
df = df.drop(df[(df.Borough == "Not assigned")].index)
df.Neighbourhood.replace("Not assigned", df.Borough, inplace=True)

In [293]:
df.shape

(210, 3)

In [294]:
df


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [295]:
# merge multiple neighbourhoods with the same Postcode
df2 = pd.DataFrame({'Postcode':df.Postcode.unique()})
df2['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=pd.Series(list(set(df['Neighbourhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=df2['Neighborhood'].apply(lambda x: ', '.join(x))
df2.dtypes

df2.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,Queen's Park


In [296]:
df2

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [297]:
### 

In [298]:
# download Zip Code csv file
dfzip = pd.read_csv("http://cocl.us/Geospatial_data")

In [299]:
# Add Lat/Long to table
toronto_data = df2.join(dfzip.set_index(['Postal Code'],verify_integrity=True),
                       on=['Postcode'],how='right')

In [300]:
toronto_data

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


## Creating Toronto map with neighbourhoods 

In [301]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [302]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

## Defining Foursquare Credentials and Version

In [303]:
CLIENT_ID = 'C3ACHZMKED50MIJFXAT1SWXPQOMS3552K4CODMSVW3T4FKLP' # your Foursquare ID
CLIENT_SECRET = 'I0XECRJGFM4Z0AOBAWHPDRPCBAF31UCT2IFVCOTURYOZNNTQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: C3ACHZMKED50MIJFXAT1SWXPQOMS3552K4CODMSVW3T4FKLP
CLIENT_SECRET:I0XECRJGFM4Z0AOBAWHPDRPCBAF31UCT2IFVCOTURYOZNNTQ


## Exploring the first neighborhood in our dataframe.

In [304]:
toronto_data

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [305]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [306]:

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=C3ACHZMKED50MIJFXAT1SWXPQOMS3552K4CODMSVW3T4FKLP&client_secret=I0XECRJGFM4Z0AOBAWHPDRPCBAF31UCT2IFVCOTURYOZNNTQ&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [307]:
results = requests.get(url).json()
#results

In [308]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [309]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114
2,TTC stop - 44 Valley Woods,Bus Stop,43.755402,-79.333741


In [310]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


## Explore Neighborhoods in Toronto

In [311]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [315]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Princess Gardens, West Deane Park, Martin Grove
Port Union, Highland Creek, Rouge Hill
Don Mills South, Flemingdon Park
Woodbine Heights
St. James Town
Humewood-Cedarvale
Old Burnhamthorpe, Eringate, Markland Wood, Bloordale Gardens
West Hill, Morningside, Guildwood
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Wilson Heights, Downsview North, Bathurst Manor
Thorncliffe Park
Adelaide, King, Richmond
Dufferin, Dovercourt Village
Scarborough Village
Henry Farm, Oriole, Fairview
Northwood Park, York University
East Toronto
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
East Birchmount Park, Kennedy Park, Ionview
Bayview Village
CFB Toronto, Downsview East
Riverdale, The Dan

In [316]:
#print(toronto_venues.shape)
#toronto_venues
#print(toronto_venues.loc['Agincourt'])

# 1. create the condition boolean series
condition = toronto_venues['Neighborhood'] == 'Agincourt'
# print(condition)
# 2. pass this condition into the dataFrame
toronto_venues[condition]

#

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1559,Agincourt,43.7942,-79.262029,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot
1560,Agincourt,43.7942,-79.262029,El Pulgarcito,43.792648,-79.259208,Latin American Restaurant
1561,Agincourt,43.7942,-79.262029,Twilight,43.791999,-79.258584,Lounge
1562,Agincourt,43.7942,-79.262029,Commander Arena,43.794867,-79.267989,Skating Rink


In [317]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, Steeles East, L'Amoreaux East, Milliken",2,2,2,2,2,2
"Alderwood, Long Branch",9,9,9,9,9,9
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,57,57,57,57,57,57
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CFB Toronto, Downsview East",4,4,4,4,4,4


In [318]:
toronto_venues.shape

(2222, 7)

In [319]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 271 uniques categories.


In [320]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
5,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
6,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
7,Harbourfront,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
8,Harbourfront,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
9,Harbourfront,43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center


## Analyzing neighborhoods

In [321]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [322]:
toronto_onehot.shape

(2222, 271)

In [323]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.01
1,Agincourt,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Agincourt North, Steeles East, L'Amoreaux East...",0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Bayview Village,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Berczy Park,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,Business Reply Mail Processing Centre 969 Eastern,0.058824,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,"CFB Toronto, Downsview East",0.000000,0.0,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,"Cabbagetown, St. James Town",0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [324]:
toronto_grouped.shape

(100, 271)

In [325]:

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0       Coffee Shop  0.07
1        Restaurant  0.05
2              Café  0.04
3   Thai Restaurant  0.04
4  Sushi Restaurant  0.03


----Agincourt----
                       venue  freq
0                     Lounge  0.25
1  Latin American Restaurant  0.25
2               Skating Rink  0.25
3             Breakfast Spot  0.25
4                Men's Store  0.00


----Agincourt North, Steeles East, L'Amoreaux East, Milliken----
                      venue  freq
0                Playground   0.5
1                      Park   0.5
2               Yoga Studio   0.0
3  Mediterranean Restaurant   0.0
4         Mobile Phone Shop   0.0


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.22
1           Pub  0.11
2      Pharmacy  0.11
3           Gym  0.11
4  Skating Rink  0.11


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2  Japanese Restaurant  0.25
3  

4     Electronics Store  0.03


----Hillcrest Village----
                      venue  freq
0      Fast Food Restaurant   0.2
1               Golf Course   0.2
2                      Pool   0.2
3                   Dog Run   0.2
4  Mediterranean Restaurant   0.2


----Humber Bay Shores, New Toronto, Mimico South----
            venue  freq
0    Liquor Store  0.07
1          Bakery  0.07
2     Pizza Place  0.07
3     Coffee Shop  0.07
4  Rental Service  0.07


----Humber Summit----
                             venue  freq
0                      Pizza Place   0.5
1              Empanada Restaurant   0.5
2  Molecular Gastronomy Restaurant   0.0
3                           Lounge   0.0
4               Mac & Cheese Joint   0.0


----Humewood-Cedarvale----
               venue  freq
0       Hockey Arena  0.25
1              Trail  0.25
2               Park  0.25
3              Field  0.25
4  Mobile Phone Shop  0.00


----King and Spadina, South Niagara, Bathurst Quay, Island airport, Railway 

                 venue  freq
0          Coffee Shop  0.12
1           Restaurant  0.04
2                 Café  0.04
3  Japanese Restaurant  0.03
4                Hotel  0.03


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.07
2  American Restaurant  0.05
3   Italian Restaurant  0.05
4            Gastropub  0.05


----Swansea, Runnymede----
                venue  freq
0                Café  0.08
1         Pizza Place  0.08
2    Sushi Restaurant  0.06
3         Coffee Shop  0.06
4  Italian Restaurant  0.06


----The Beaches----
               venue  freq
0  Health Food Store  0.25
1                Pub  0.25
2              Trail  0.25
3        Yoga Studio  0.00
4        Men's Store  0.00


----The Beaches West, India Bazaar----
                  venue  freq
0             Pet Store  0.06
1         Movie Theater  0.06
2          Liquor Store  0.06
3  Fast Food Restaurant  0.06
4                  Park  0.06


----The Junction North

In [326]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [327]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Restaurant,Thai Restaurant,Café,Bar,Sushi Restaurant,Gastropub,Hotel,Lounge,Gym
1,Agincourt,Skating Rink,Lounge,Breakfast Spot,Latin American Restaurant,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,"Agincourt North, Steeles East, L'Amoreaux East...",Park,Playground,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
3,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Pool,Skating Rink,Gym,Pharmacy,Pub,Sandwich Place,Dance Studio,Deli / Bodega
4,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store


In [328]:
print(toronto_venues.shape)
toronto_venues.head()

(2222, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


## Clustering Neighborhoods

In [329]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 4, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [330]:
toronto_grouped_clustering

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.020000,...,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.01
1,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.043478,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,0.058824,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,0.000000,0.0,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [331]:
toronto_data = neighborhoods[['Borough','Neighborhood','Latitude','Longitude']]
toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,Scarborough,"Port Union, Highland Creek, Rouge Hill",43.784535,-79.160497
2,Scarborough,"West Hill, Morningside, Guildwood",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [332]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Restaurant,Thai Restaurant,Café,Bar,Sushi Restaurant,Gastropub,Hotel,Lounge,Gym
1,Agincourt,Skating Rink,Lounge,Breakfast Spot,Latin American Restaurant,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,"Agincourt North, Steeles East, L'Amoreaux East...",Park,Playground,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
3,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Pool,Skating Rink,Gym,Pharmacy,Pub,Sandwich Place,Dance Studio,Deli / Bodega
4,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store
5,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Restaurant,Pizza Place,Butcher,Pub,Sushi Restaurant,Indian Restaurant,Café
6,Berczy Park,Coffee Shop,Farmers Market,Cheese Shop,Restaurant,Café,Seafood Restaurant,Bakery,Cocktail Bar,Beer Bar,Shopping Mall
7,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Pizza Place,Restaurant,Butcher,Light Rail Station,Auto Workshop,Fast Food Restaurant,Farmers Market,Burrito Place,Spa
8,"CFB Toronto, Downsview East",Park,Snack Place,Airport,Construction & Landscaping,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
9,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Park,Pizza Place,Italian Restaurant,Pub,Bakery,Café,Beer Store,Pharmacy


In [333]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [334]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Adelaide, King, Richmond",Coffee Shop,Restaurant,Thai Restaurant,Café,Bar,Sushi Restaurant,Gastropub,Hotel,Lounge,Gym
1,0,Agincourt,Skating Rink,Lounge,Breakfast Spot,Latin American Restaurant,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,4,"Agincourt North, Steeles East, L'Amoreaux East...",Park,Playground,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
3,0,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Pool,Skating Rink,Gym,Pharmacy,Pub,Sandwich Place,Dance Studio,Deli / Bodega
4,0,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store
5,0,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Restaurant,Pizza Place,Butcher,Pub,Sushi Restaurant,Indian Restaurant,Café
6,0,Berczy Park,Coffee Shop,Farmers Market,Cheese Shop,Restaurant,Café,Seafood Restaurant,Bakery,Cocktail Bar,Beer Bar,Shopping Mall
7,0,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Pizza Place,Restaurant,Butcher,Light Rail Station,Auto Workshop,Fast Food Restaurant,Farmers Market,Burrito Place,Spa
8,0,"CFB Toronto, Downsview East",Park,Snack Place,Airport,Construction & Landscaping,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
9,0,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Park,Pizza Place,Italian Restaurant,Pub,Bakery,Café,Beer Store,Pharmacy


In [335]:
#neighborhoods_venues_sorted.dtypes
#torontomerged
#toronto_data.dtypes
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#kmeans.labels_
#toronto_grouped.dtypes

In [336]:
# add clustering labels
toronto_merged = toronto_data
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
1,Scarborough,"Port Union, Highland Creek, Rouge Hill",43.784535,-79.160497,0.0,Bar,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Ethiopian Restaurant
2,Scarborough,"West Hill, Morningside, Guildwood",43.763573,-79.188711,0.0,Electronics Store,Breakfast Spot,Mexican Restaurant,Medical Center,Intersection,Rental Car Location,Spa,Dog Run,Doner Restaurant,Distribution Center
3,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
4,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Bakery,Caribbean Restaurant,Bank,Lounge,Hakka Restaurant,Fried Chicken Joint,Athletics & Sports,Gas Station,Thai Restaurant,Department Store


In [337]:
toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
1,Scarborough,"Port Union, Highland Creek, Rouge Hill",43.784535,-79.160497,0.0,Bar,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Ethiopian Restaurant
2,Scarborough,"West Hill, Morningside, Guildwood",43.763573,-79.188711,0.0,Electronics Store,Breakfast Spot,Mexican Restaurant,Medical Center,Intersection,Rental Car Location,Spa,Dog Run,Doner Restaurant,Distribution Center
3,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
4,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Bakery,Caribbean Restaurant,Bank,Lounge,Hakka Restaurant,Fried Chicken Joint,Athletics & Sports,Gas Station,Thai Restaurant,Department Store


In [338]:
toronto_merged.dtypes

Borough                    object
Neighborhood               object
Latitude                  float64
Longitude                 float64
Cluster Labels            float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [339]:
toronto_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
1,Scarborough,"Port Union, Highland Creek, Rouge Hill",43.784535,-79.160497,0.0,Bar,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Ethiopian Restaurant
2,Scarborough,"West Hill, Morningside, Guildwood",43.763573,-79.188711,0.0,Electronics Store,Breakfast Spot,Mexican Restaurant,Medical Center,Intersection,Rental Car Location,Spa,Dog Run,Doner Restaurant,Distribution Center
3,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
4,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Bakery,Caribbean Restaurant,Bank,Lounge,Hakka Restaurant,Fried Chicken Joint,Athletics & Sports,Gas Station,Thai Restaurant,Department Store
5,Scarborough,Scarborough Village,43.744734,-79.239476,4.0,Playground,Women's Store,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
6,Scarborough,"East Birchmount Park, Kennedy Park, Ionview",43.727929,-79.262029,0.0,Department Store,Coffee Shop,Discount Store,Chinese Restaurant,Convenience Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
7,Scarborough,"Clairlea, Oakridge, Golden Mile",43.711112,-79.284577,0.0,Bakery,Bus Line,Bus Station,Intersection,Ice Cream Shop,Soccer Field,Metro Station,Dim Sum Restaurant,Diner,Discount Store
8,Scarborough,"Scarborough Village West, Cliffcrest, Cliffside",43.716316,-79.239476,0.0,Motel,American Restaurant,Women's Store,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
9,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848,0.0,Skating Rink,College Stadium,General Entertainment,Café,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner


In [340]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].replace(np.nan, 5)

In [341]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].apply(np.int64)

In [342]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining Clusters

### Cluster 1

In [343]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",Fast Food Restaurant,Print Shop,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
1,"Port Union, Highland Creek, Rouge Hill",Bar,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Ethiopian Restaurant
2,"West Hill, Morningside, Guildwood",Electronics Store,Breakfast Spot,Mexican Restaurant,Medical Center,Intersection,Rental Car Location,Spa,Dog Run,Doner Restaurant,Distribution Center
3,Woburn,Coffee Shop,Korean Restaurant,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
4,Cedarbrae,Bakery,Caribbean Restaurant,Bank,Lounge,Hakka Restaurant,Fried Chicken Joint,Athletics & Sports,Gas Station,Thai Restaurant,Department Store
6,"East Birchmount Park, Kennedy Park, Ionview",Department Store,Coffee Shop,Discount Store,Chinese Restaurant,Convenience Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
7,"Clairlea, Oakridge, Golden Mile",Bakery,Bus Line,Bus Station,Intersection,Ice Cream Shop,Soccer Field,Metro Station,Dim Sum Restaurant,Diner,Discount Store
8,"Scarborough Village West, Cliffcrest, Cliffside",Motel,American Restaurant,Women's Store,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
9,"Cliffside West, Birch Cliff",Skating Rink,College Stadium,General Entertainment,Café,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
10,"Scarborough Town Centre, Wexford Heights, Dors...",Indian Restaurant,Pet Store,Vietnamese Restaurant,Gaming Cafe,Chinese Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Curling Ice


### Cluster 2

In [344]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,"Silver Hills, York Mills",Cafeteria,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dance Studio


### Cluster 3

In [345]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Downsview Central,Food Truck,Home Service,Baseball Field,Women's Store,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
63,Roselawn,Home Service,Garden,Women's Store,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
91,"Mimico NE, Sunnylea, Kingsway Park South East,...",Home Service,Baseball Field,Women's Store,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
94,"Cloverdale, Islington, Princess Gardens, West ...",Home Service,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dance Studio


### Cluster 4

In [346]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,York Mills West,Park,Bank,Convenience Store,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
25,Parkwoods,Park,Food & Drink Shop,Bus Stop,Women's Store,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Distribution Center
40,East Toronto,Park,Convenience Store,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
44,Lawrence Park,Park,Bus Line,Swim School,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center,Dance Studio
48,"Moore Park, Summerhill East",Restaurant,Park,Gym,Tennis Court,Diner,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
74,Caledonia-Fairbanks,Park,Women's Store,Market,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dance Studio,Dog Run
90,"Old Mill North, Montgomery Road, The Kingsway",Park,Pool,River,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Cupcake Shop
98,Weston,Park,Convenience Store,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


### Cluster 5

In [347]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough Village,Playground,Women's Store,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
14,"Agincourt North, Steeles East, L'Amoreaux East...",Park,Playground,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
50,Rosedale,Park,Trail,Playground,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
